In [64]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, PredefinedSplit
from sklearn.feature_extraction import DictVectorizer
from scipy.stats import uniform, randint
from helpers import *

In [65]:
# path to project directory
path = Path('./')

In [66]:
# read in training dataset
train_df = pd.read_csv(path/'data/train_v4.csv', index_col=0, dtype={'season':str})

## XGBoost model

XGboost is a ensemble tree-based predictive algorithm that performs well across a range of applications. Applying it to a time series problem, where metrics from recent time periods can be predicitve, requires us to add in window features (e.g. points scored last gameweek). These are created using the player_lag_features function from 00_fpl_features.

In [67]:
# add a bunch of player lag features
lag_train_df, player_lag_vars = player_lag_features(train_df, ['total_points'], ['all', 1, 2, 3, 4, 5, 10])

We have introduced a number of lag (window) features for each player's average points per game over the previous 1, 2, 3, 4, 5, 10 and all gameweeks.

Next we can set the validation point and length as well as the categorical and continuous features we'll be using to predict the dependent variable, total points for each game. These are used in the create_lag_train function to get an our training set (including appropriate lag values in the validation set).

In [69]:
# set validaton point/length and categorical/continuous variables
valid_season = '1920'
valid_gw = 20
valid_len = 6
cat_vars = ['season', 'position', 'team', 'opponent_team', 'was_home']
cont_vars = ['gw', 'minutes']
dep_var = ['total_points']

# create dataset with adjusted post-validation lag numbers
lag_train_df, train_idx, valid_idx = create_lag_train(lag_train_df, 
                                                      cat_vars, cont_vars, player_lag_vars, dep_var,
                                                      valid_season, valid_gw, valid_len)

In [70]:
# take a look at the dataframe
lag_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82054 entries, 0 to 82053
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   index                     82054 non-null  int64  
 1   player                    82054 non-null  object 
 2   season                    82054 non-null  object 
 3   position                  82054 non-null  int64  
 4   team                      82054 non-null  object 
 5   opponent_team             82054 non-null  object 
 6   was_home                  82054 non-null  bool   
 7   gw                        82054 non-null  int64  
 8   minutes                   82054 non-null  int64  
 9   total_points_pg_last_all  68666 non-null  float64
 10  total_points_pg_last_1    37879 non-null  float64
 11  total_points_pg_last_2    44333 non-null  float64
 12  total_points_pg_last_3    47807 non-null  float64
 13  total_points_pg_last_4    50292 non-null  float64
 14  total_

We can now build the input (X) and dependent (y) variable datasets. This includes encoding the categorical features so that each level is represented in it's own column (e.g. postition_1, position_2, etc.).

In [21]:
# split out dependent variable
X, y = lag_train_df[cat_vars + cont_vars + player_lag_vars].copy(), lag_train_df[dep_var].copy()

In [22]:
# since position is categorical, it should be a string
X['position'] = X['position'].apply(str)

# need to transform season
enc = LabelEncoder()
X['season'] = enc.fit_transform(X['season'])
X_dict = X.to_dict("records")

# Create the DictVectorizer object: dv
dv = DictVectorizer(sparse=False, separator='_')

# Apply dv on df: df_encoded
X_encoded = dv.fit_transform(X_dict)

X_df = pd.DataFrame(X_encoded, columns=dv.feature_names_)

You can see the resulting dataset with the categorical features split out into levels. This can now be used in the XGBoost API.

In [26]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82054 entries, 0 to 82053
Data columns (total 73 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   gw                                      82054 non-null  float64
 1   minutes                                 82054 non-null  float64
 2   opponent_team_Arsenal                   82054 non-null  float64
 3   opponent_team_Aston Villa               82054 non-null  float64
 4   opponent_team_Bournemouth               82054 non-null  float64
 5   opponent_team_Brighton and Hove Albion  82054 non-null  float64
 6   opponent_team_Burnley                   82054 non-null  float64
 7   opponent_team_Cardiff City              82054 non-null  float64
 8   opponent_team_Chelsea                   82054 non-null  float64
 9   opponent_team_Crystal Palace            82054 non-null  float64
 10  opponent_team_Everton                   82054 non-null  fl

In [16]:
X[player_lag_vars].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82054 entries, 0 to 82053
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   total_points_pg_last_all  68666 non-null  float64
 1   total_points_pg_last_1    37879 non-null  float64
 2   total_points_pg_last_2    44333 non-null  float64
 3   total_points_pg_last_3    47807 non-null  float64
 4   total_points_pg_last_4    50292 non-null  float64
 5   total_points_pg_last_5    52087 non-null  float64
 6   total_points_pg_last_10   57642 non-null  float64
dtypes: float64(7)
memory usage: 4.4 MB


We can start by instatiating an XGBRegressor (since the dependent variable is continuous) and do a single train with arbitrary parameters. We split out the validation set and use it after training to create predictions and calculate the RMSE versus actuals.

In [17]:
# split out training and validation sets
X_train = X_df.iloc[train_idx]
y_train = y.iloc[train_idx]
X_test = X_df.iloc[valid_idx]
y_test = y.iloc[valid_idx]

In [18]:
# instatiate and train XGB Regressor
# print result
xg_reg = xgb.XGBRegressor(gamma=0.05, learning_rate=0.08, max_depth=5, n_estimators=75, subsample=0.7)

xg_reg.fit(X_train, y_train)
preds = xg_reg.predict(X_test)
print("RMSE: %f" % (r_mse(preds, y_test['total_points'])))

RMSE: 1.791749


This is a clear improvement on the baseline approach, but perhaps it can be improved by doing a parameter search.

To do this we will first define the grid of parameters to be searched.

In [28]:
# parameter search space
params = {#"colsample_bytree": uniform(0.7, 0.3),
          "gamma": uniform(0, 0.5),
          "learning_rate": uniform(0.003, 0.3), # default 0.1 
          "max_depth": randint(2, 6), # default 3
          "n_estimators": randint(25, 200), # default 100
          "subsample": uniform(0.6, 0.4)}

In this case we will pass both train and validation parts of the dataset, along with a series telling the XGBRegressor object which rows to use for training, and which for validation.

In [61]:
X_train = X_df
y_train = y
test_fold = np.repeat([-1, 0], [valid_idx[0], valid_idx[-1] - valid_idx[0] + 1])
ps = PredefinedSplit(test_fold)

We can then again instatiate the XGBRegressor object, but this time pass it to a randomised search validation object, along with the parameter grid, validation splits, and number of iterations we want to run.

We then fit this to the training data - 25 random parameter selections will be made and the best parameters for the validation set can be found (may take a few minutes to run).

In [63]:
# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(objective="reg:squarederror")

# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator=gbm, 
                            param_distributions=params, 
                            scoring="neg_mean_squared_error", 
                            n_iter=25, 
                            cv=ps, 
                            verbose=1)

# Fit randomized_mse to the data
randomized_mse.fit(X_train, y_train)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

Fitting 1 folds for each of 25 candidates, totalling 25 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  8.1min finished


Best parameters found:  {'gamma': 0.2819157508996368, 'learning_rate': 0.12981570144081725, 'max_depth': 3, 'n_estimators': 174, 'subsample': 0.9465078528713007}
Lowest RMSE found:  1.7893024515231921


A slight improvement on the above.

In [ ]:
# Create the parameter grid: gbm_param_grid 
gbm_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 5],
    #'colsample_bytree': [0.1, 0.5, 0.8, 1],
    'learning_rate': [0.1]
}